In [330]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

seed = 1234
tf.random.set_seed(seed)
np.random.seed(seed)

In [331]:
data_positive = pd.read_csv("samples_positive.csv")
data_negative = pd.read_csv("samples_negative.csv")

In [332]:
data = pd.concat([data_positive, data_negative])
data.head()

,GPA,SAT Reading & Writing,SAT Math,SAT Essay,Activity Score,Personal Statement Score,Residency,Race,Gender,Accepted
0,4.0,751,709,17,80,80,International,Asian,Female,1
1,3.7,790,782,17,80,80,CA resident,Asian,Male,1
2,3.5,744,729,16,80,80,International,White,Male,1
3,4.0,759,737,17,80,80,CA resident,White,Female,1
4,4.0,707,759,18,80,80,CA resident,Asian,Female,1


In [333]:
obj_cols = data.select_dtypes(include=['object']).columns
processed_data = data.copy()
residency_ordinal_encoder = OrdinalEncoder()
processed_data[obj_cols] = residency_ordinal_encoder.fit_transform(data[obj_cols])

In [334]:
processed_data.head()

,GPA,SAT Reading & Writing,SAT Math,SAT Essay,Activity Score,Personal Statement Score,Residency,Race,Gender,Accepted
0,4.0,751,709,17,80,80,1.0,2.0,0.0,1
1,3.7,790,782,17,80,80,0.0,2.0,2.0,1
2,3.5,744,729,16,80,80,1.0,7.0,2.0,1
3,4.0,759,737,17,80,80,0.0,7.0,0.0,1
4,4.0,707,759,18,80,80,0.0,2.0,0.0,1


In [335]:
shuffled_data = processed_data.sample(frac=1, random_state=seed).reset_index(drop=True)

In [336]:
shuffled_data.head()

,GPA,SAT Reading & Writing,SAT Math,SAT Essay,Activity Score,Personal Statement Score,Residency,Race,Gender,Accepted
0,3.6,751,753,16,80,80,0.0,2.0,0.0,1
1,3.9,712,796,16,80,80,0.0,2.0,2.0,1
2,3.6,768,743,18,80,80,1.0,4.0,0.0,1
3,3.6,717,783,16,80,80,0.0,4.0,2.0,1
4,3.9,785,744,16,80,80,0.0,2.0,0.0,1


In [337]:
y = shuffled_data.pop("Accepted")
X = shuffled_data.copy()

In [338]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [339]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(9, ))
    ])
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [340]:
model = create_model()

In [341]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

Epoch 1/50
150/150 [==============================] - 0s 1ms/step - loss: 1085.5012 - accuracy: 0.5619 - val_loss: 1199.7026 - val_accuracy: 0.6567
Epoch 2/50
150/150 [==============================] - 0s 893us/step - loss: 1053.5315 - accuracy: 0.5660 - val_loss: 360.6918 - val_accuracy: 0.6567
Epoch 3/50
150/150 [==============================] - 0s 905us/step - loss: 1031.7712 - accuracy: 0.5640 - val_loss: 2148.1509 - val_accuracy: 0.3433
Epoch 4/50
150/150 [==============================] - 0s 890us/step - loss: 926.9409 - accuracy: 0.5919 - val_loss: 11.2688 - val_accuracy: 0.9442
Epoch 5/50
150/150 [==============================] - 0s 924us/step - loss: 956.3192 - accuracy: 0.5823 - val_loss: 574.1177 - val_accuracy: 0.3433
Epoch 6/50
150/150 [==============================] - 0s 1ms/step - loss: 858.4219 - accuracy: 0.5925 - val_loss: 1827.5643 - val_accuracy: 0.6567
Epoch 7/50
150/150 [==============================] - 0s 931us/step - loss: 899.6639 - accuracy: 0.5921 - val_l

In [342]:
model.predict([[3.5, 710, 740, 18, 80, 80, 3.0, 5.0, 0.0], 
               [4.0, 780, 780, 18, 80, 80, 3.0, 5.0, 0.0]])

1/1 [==============================] - 0s 45ms/step


array([[1.],
       [1.]], dtype=float32)

In [343]:
model.predict([[2.0, 600, 600, 16, 100, 100, 3.0, 2.0, 0.0],
               [2.6, 700, 600, 13, 80, 80, 1.0, 2.0, 1.0]])

1/1 [==============================] - 0s 26ms/step


array([[0.],
       [0.]], dtype=float32)

In [344]:
model.save_weights("logistic_weights.h5")

In [345]:
model = create_model()

In [346]:
model.load_weights("logistic_weights.h5")

In [347]:
model.evaluate(X_test, y_test)

38/38 [==============================] - 0s 623us/step - loss: 46.7359 - accuracy: 0.9375


[46.73590850830078, 0.9375]